In [0]:
sc


SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
print(spark)

In [0]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import *

##### Q1) Create DataFrame using Read CSV function

In [0]:
Airlines = spark.read.csv("dbfs:/FileStore/tables/Flights_Delay.csv", header = True, inferSchema = True)

##### Q2) Describe the table schema and show 10 rows of datasets

In [0]:
Airlines.show(10, truncate = False)

+---+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|ID |YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+---+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+-----------

##### Q3) Average arrival delay caused by airlines

####### average arrival delay of specific airlines

In [0]:
Airlines.groupBy("AIRLINE").mean("ARRIVAL_DELAY").show()


+-------+------------------+
|AIRLINE|avg(ARRIVAL_DELAY)|
+-------+------------------+
|     UA| 6.697221614526362|
|     NK|14.206426484907498|
|     AA| 8.386631979187513|
|     EV|10.884270870655678|
|     B6| 13.95852534562212|
|     DL|2.8144726712856043|
|     OO|10.154792043399638|
|     F9|24.103448275862068|
|     US| 5.977315185481719|
|     MQ|19.231592604605904|
|     HA| 4.072423398328691|
|     AS|-1.531766200762389|
|     VX| 5.128571428571429|
|     WN| 3.697840458351697|
+-------+------------------+



##### Q4) Days of months with average arrival delay

In [0]:
Airlines.createTempView("AirlineTable")


In [0]:
spark.sql("select DAY, AVG(ARRIVAL_DELAY) as avgDelay from AirlineTable group by DAY order by avgDelay desc").show()

+---+------------------+
|DAY|          avgDelay|
+---+------------------+
|  3|18.141541038525965|
|  4|17.157790927021697|
|  5| 16.23861262014208|
|  2|15.046014790468364|
|  1|14.807807807807809|
| 26| 11.96778269109286|
| 12| 11.24892703862661|
|  6|10.608832807570979|
| 16| 9.124321062160531|
| 17| 8.761435608726249|
| 21| 6.919860627177701|
| 22| 6.550920245398773|
| 24| 5.737543859649123|
|  8| 5.232349165596919|
| 25| 4.903708523096942|
| 27| 4.706711409395973|
| 30| 4.471478463329452|
|  9| 4.421887390959556|
| 23| 4.207086133170434|
| 11|3.9912935323383083|
+---+------------------+
only showing top 20 rows



##### Q5) Days of Month with median departure delay

In [0]:
spark.sql("select DAY, PERCENTILE_APPROX(DEPARTURE_DELAY, 0.5) as medianDelay from AirlineTable group by DAY order by medianDelay desc").show()

+---+-----------+
|DAY|medianDelay|
+---+-----------+
|  2|          2|
|  5|          1|
|  3|          1|
| 12|          0|
|  6|          0|
| 17|          0|
|  1|          0|
|  4|          0|
| 16|         -1|
| 15|         -1|
|  9|         -1|
| 23|         -1|
| 26|         -1|
|  8|         -1|
| 22|         -1|
| 13|         -1|
| 28|         -2|
| 27|         -2|
| 20|         -2|
| 19|         -2|
+---+-----------+
only showing top 20 rows



##### Q6) Arrange weekdays with respect to average arrival delays

In [0]:
spark.sql("select DAY_OF_WEEK, AVG(ARRIVAL_DELAY) as avgDelay from AirlineTable group by DAY_OF_WEEK order by avgDelay desc").show()

+-----------+------------------+
|DAY_OF_WEEK|          avgDelay|
+-----------+------------------+
|          1|10.807447207297264|
|          7|10.110840438489646|
|          2| 8.033644102148358|
|          4| 7.174969021065675|
|          5| 6.010538373424971|
|          3| 5.587079407806191|
|          6| 4.888689138576779|
+-----------+------------------+



##### Q7) Show analysis for each month with total number of cancellation

In [0]:
spark.sql("select MONTH, SUM(if(CANCELLED=1,1,0)) as Total_Flight_Cancelled from AirlineTable group by MONTH order by Total_Flight_Cancelled desc").show()

+-----+----------------------+
|MONTH|Total_Flight_Cancelled|
+-----+----------------------+
|    2|                  1087|
|    1|                   608|
|    3|                   456|
+-----+----------------------+



##### Q8) Show top 10 busiest airport

In [0]:
spark.sql("select ORIGIN_AIRPORT, SUM(DEPARTURE_TIME + ARRIVAL_TIME) as busy from AirlineTable group by ORIGIN_AIRPORT order by busy desc").show(10)

+--------------+--------+
|ORIGIN_AIRPORT|    busy|
+--------------+--------+
|           ATL|10600667|
|           ORD| 8037828|
|           DFW| 7894514|
|           DEN| 5863791|
|           LAX| 5709570|
|           IAH| 4697727|
|           PHX| 4548864|
|           SFO| 4171803|
|           LAS| 3910375|
|           MCO| 3357189|
+--------------+--------+
only showing top 10 rows



##### Q9) show the airline that make maxuimum number of cancellation

In [0]:
spark.sql("select AIRLINE, SUM(CANCELLED) as flight_cancelled from AirlineTable group by AIRLINE order by flight_cancelled desc").show(1)

+-------+----------------+
|AIRLINE|flight_cancelled|
+-------+----------------+
|     MQ|             414|
+-------+----------------+
only showing top 1 row



##### Q10) find and order airlines in descending that makes the most number of diversions

In [0]:
spark.sql("select AIRLINE, SUM(DIVERTED) as flight_diverted from AirlineTable group by AIRLINE order by flight_diverted desc").show()

+-------+---------------+
|AIRLINE|flight_diverted|
+-------+---------------+
|     WN|             35|
|     OO|             25|
|     EV|             22|
|     DL|             18|
|     B6|             16|
|     AA|             12|
|     US|              9|
|     UA|              8|
|     MQ|              5|
|     HA|              1|
|     NK|              0|
|     F9|              0|
|     AS|              0|
|     VX|              0|
+-------+---------------+



##### Q11) Show days of months that have seen more number of diversions

In [0]:
spark.sql("select DAY, SUM(DIVERTED) as flight_diverted from AirlineTable group by DAY order by flight_diverted desc").show(1)

+---+---------------+
|DAY|flight_diverted|
+---+---------------+
|  2|             15|
+---+---------------+
only showing top 1 row



##### Q12) Calculate mean and standard deviation of departure delay for all flights in minutes.

In [0]:
spark.sql("select AIRLINE, AVG(DEPARTURE_DELAY) as AvgDepDelay, STDDEV(DEPARTURE_DELAY) as devDepDelay from AirlineTable group by AIRLINE ").show()

+-------+------------------+------------------+
|AIRLINE|       AvgDepDelay|       devDepDelay|
+-------+------------------+------------------+
|     UA|14.293129770992367| 36.36679305829193|
|     NK|15.580739299610896|46.004490473829854|
|     AA|11.502293120638086| 50.58770621277648|
|     EV|11.534709861160556|  40.6022135127612|
|     B6|16.068664169787766|44.452210632841826|
|     DL| 9.938812083973374| 44.57836834851712|
|     OO|11.601510519690702|41.898251511636026|
|     F9|23.514687100893997| 55.21644901488807|
|     US| 7.806271591815041|29.947392168792227|
|     MQ| 17.07297384565709| 43.46879359686762|
|     HA|1.1835883171070931| 30.28223244986787|
|     AS|2.3365079365079366| 29.14656093746487|
|     VX| 9.857142857142858|35.175272138214424|
|     WN|10.115161630358397| 28.66211787470035|
+-------+------------------+------------------+



##### Q13) Calculate mean and standard deviation of Arrival delay for all flights in minutes.

In [0]:
spark.sql("select AIRLINE, AVG(ARRIVAL_DELAY) as AvgArrDelay, STDDEV(ARRIVAL_DELAY) as devArrDelay from AirlineTable group by AIRLINE ").show()

+-------+------------------+------------------+
|AIRLINE|       AvgArrDelay|       devArrDelay|
+-------+------------------+------------------+
|     UA| 6.697221614526362| 38.96583521422204|
|     NK|14.206426484907498| 47.57688184145277|
|     AA| 8.386631979187513| 53.56770860254512|
|     EV|10.884270870655678| 43.39199993434066|
|     B6| 13.95852534562212| 47.64386997495008|
|     DL|2.8144726712856043| 46.95730281739299|
|     OO|10.154792043399638| 43.76036682697078|
|     F9|24.103448275862068| 56.26804479748733|
|     US| 5.977315185481719| 34.10900474893441|
|     MQ|19.231592604605904| 46.39998574481305|
|     HA| 4.072423398328691| 32.38330048967069|
|     AS|-1.531766200762389| 31.37282637924461|
|     VX| 5.128571428571429| 40.86556219200457|
|     WN| 3.697840458351697|31.225094725977485|
+-------+------------------+------------------+



##### Q14) Find all diverted Route from a source to destination Airport & which route is the most diverted

In [0]:
spark.sql("select ORIGIN_AIRPORT, DESTINATION_AIRPORT, sum(DIVERTED) as maxDiverted from AirlineTable group by ORIGIN_AIRPORT, DESTINATION_AIRPORT order by maxDiverted desc").show()

+--------------+-------------------+-----------+
|ORIGIN_AIRPORT|DESTINATION_AIRPORT|maxDiverted|
+--------------+-------------------+-----------+
|           IAH|                ASE|          2|
|           STT|                PHL|          2|
|           PHL|                SAN|          2|
|           HOU|                DAL|          2|
|           ORD|                ASE|          2|
|           JFK|                EGE|          2|
|           CLT|                IAH|          2|
|           JFK|                SEA|          2|
|           TPA|                LGA|          2|
|           MCO|                PVD|          1|
|           ATL|                ASE|          1|
|           FLL|                PVD|          1|
|           ATL|                LGA|          1|
|           IAH|                ISN|          1|
|           KOA|                SFO|          1|
|           SLC|                RDM|          1|
|           SBP|                SFO|          1|
|           LAX|    

##### Q15) Finding AIRLINES with its total flight count, total number of flights arrival delayed by more than 30 Minutes, % of such flights delayed by more than 30 minutes when it is not Weekends with minimum count of flights from Airlines by more than 10. Also Exclude some of Airlines 'AK', 'HI', 'PR', 'VI' and arrange output in descending order by % of such count of flights.

In [0]:
spark.sql("select AIRLINE, count(*) as total_counts, SUM(if(ARRIVAL_DELAY>30,1,0)) as totaldelaymorethan30mins,  SUM(if(ARRIVAL_DELAY>30,1,0))/count(*)*100 as percent from AirlineTable group by AIRLINE order by percent desc").show()

+-------+------------+------------------------+------------------+
|AIRLINE|total_counts|totaldelaymorethan30mins|           percent|
+-------+------------+------------------------+------------------+
|     F9|         794|                     198|24.937027707808564|
|     MQ|        3502|                     775|22.130211307824098|
|     B6|        2548|                     485| 19.03453689167975|
|     NK|        1048|                     186|17.748091603053435|
|     OO|        5708|                     859|15.049053959355291|
|     EV|        5916|                     874|14.773495605138606|
|     UA|        4701|                     653|13.890661561369921|
|     AA|        5250|                     700|13.333333333333334|
|     VX|         573|                      67| 11.69284467713787|
|     US|        3925|                     452|11.515923566878982|
|     WN|       11738|                    1235|10.521383540637247|
|     DL|        7989|                     746|  9.33783952935